In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import scipy
import sklearn
import math
from prettytable import PrettyTable
from functools import partial

plt.rcParams["font.sans-serif"] = ["Microsoft YaHei"]  # 黑体
plt.rcParams["axes.unicode_minus"] = False  # 解决无法显示符号的问题
palette = "deep"
sns.set(font="Microsoft YaHei", font_scale=1.2, palette=palette)  # 解决Seaborn中文显示问题

In [2]:
alpha = 1.5
theta = 120
中心点海水深度 = 70

In [3]:
def 计算海水深度(距离测线中心点的距离: float) -> float:
    return 中心点海水深度 - 距离测线中心点的距离 * math.tan(math.radians(alpha))


def 计算覆盖宽度(海水深度: float) -> float:
    val = (海水深度 * math.cos(math.radians(alpha)) * math.sin(math.radians(theta))) / (
        math.cos(math.radians(alpha - theta / 2))
        * math.cos(math.radians(alpha + theta / 2))
    )
    return val


def 计算重叠率(覆盖宽度: float) -> float:
    d = 200
    val = 1 - (d / 覆盖宽度)
    return val


def 计算重叠率2(深度列表: float, 覆盖宽度: float) -> float:
    前一个深度, 当前深度 = 深度列表
    if 前一个深度 is None:
        return None
    d = 200

    val = math.sin(math.radians(theta / 2)) * (
        前一个深度 / math.cos(math.radians(alpha - theta / 2))
        + 当前深度 / math.cos(math.radians(alpha + theta / 2))
    ) - d / math.cos(math.radians(alpha))
    val = val / 覆盖宽度
    return val

In [4]:
距离测线中心点的距离 = [-800, -600, -400, -200, 0, 200, 400, 600, 800]
海水深度 = list(map(计算海水深度, 距离测线中心点的距离))
覆盖宽度 = list(map(计算覆盖宽度, 海水深度))
重叠率 = list(map(计算重叠率, 覆盖宽度))


def 转二元组(x):
    tmp = [None] + list(x)
    result = []
    for idx, val in enumerate(tmp):
        if val is None:
            continue
        result.append((tmp[idx - 1], val))

    return result


重叠率2 = map(计算重叠率2, 转二元组(海水深度), 覆盖宽度)

In [5]:
data = {
    "距离测线中心点的距离": 距离测线中心点的距离,
    "海水深度": 海水深度,
    "覆盖宽度": 覆盖宽度,
    "重叠率": 重叠率,
    "重叠率2": 重叠率2,
}

In [6]:
df = pd.DataFrame(data)
df.T

,0,1,2,3,4,5,6,7,8
距离测线中心点的距离,-800.000000,-600.000000,-400.000000,-200.000000,0.000000,200.000000,400.000000,600.000000,800.000000
海水深度,90.948737,85.711553,80.474369,75.237184,70.000000,64.762816,59.525631,54.288447,49.051263
覆盖宽度,315.813328,297.627561,279.441793,261.256025,243.070257,224.884490,206.698722,188.512954,170.327187
重叠率,0.366715,0.328019,0.284287,0.234467,0.177193,0.110655,0.032408,-0.060935,-0.174211
重叠率2,NaN,0.356954,0.315106,0.267431,0.212622,0.148949,0.074072,-0.015252,-0.123650


In [7]:
df.T.to_excel("第一问ans.xlsx")